# MLOps Workshop - Python Functions

In questo notebook andremo a fare il deploy di una Python function che processa i dati passati e li invia ad un modello di Machine Learning già in deployment.

In [ ]:
# Installazione libreria necessaria
!pip install wget
!pip install pillow 

# Setup

Installazione ed import delle librerie.

In [ ]:
# Import librerie
import json
import os
import requests
import os, wget, json
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.core.display import display, HTML

In [ ]:
cpdtoken=os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
    "token": cpdtoken,
    "instance_id" : "openshift",
    "url": os.environ['RUNTIME_ENV_APSX_URL'],
    "version": "4.0"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

# Setup Deployment

Impostiamo il deployment space ID e il deployment ID del modello da aggiungere all'interno della funzione.

In [ ]:
client.spaces.list(limit=10)

In [ ]:
# TODO Change ID

space_id=''

client.set.default_space(space_id)

In [ ]:
client.deployments.list()

In [ ]:
# TODO Change ID

model_deployment_id = ""

## Data setup

Scarichiamo un'immagine di esempio per iniziare ad esplorare i dati.

In [ ]:
# Download del file di esempio e caricamento in una variabile

sample_canvas_data_filename = 'mnist-html-canvas-image-data.json'
url = 'https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/mnist/' + sample_canvas_data_filename
if not os.path.isfile(sample_canvas_data_filename): wget.download(url)

with open(sample_canvas_data_filename) as data_file:
    sample_payload = json.load(data_file)

In [ ]:
# Plot dell'immagine con matplotlib

print("Height (n): " + str(sample_payload["height"]) + " pixels\n")
print("Num image data entries: " + str(len( sample_payload["data"])) + " - (n * n * 4) elements - RGBA values\n")

rgba_arr = np.asarray(sample_payload["data"]).astype('uint8')
n = sample_payload["height"]
plt.figure()
plt.imshow( rgba_arr.reshape(n, n, 4))
plt.xticks([])
plt.yticks([])
plt.show()

# Definzione Python Function

Definiamo la Python function di cui fare il deploy. 

Possiamo definire altre funzioni di utilità, ma l'unica funzione **obbligatoria** da inserire è la `score`, la quale viene eseguita all'invocazione dall'esterno e restituisce il risultato della funzione. 

In [ ]:
ai_parms = {"wml_credentials": wml_credentials, "space_id": space_id, "model_deployment_id": model_deployment_id}

def my_deployable_function( parms=ai_parms ):
    
    
        
    def getRGBAArr(canvas_data):
        import numpy as np
        dimension = canvas_data["height"]
        rgba_data = canvas_data["data"]
        rgba_arr  = np.asarray(rgba_data).astype('uint8')
        return rgba_arr.reshape(dimension, dimension, 4)
        
    def getNormAlphaList(img):
        import numpy as np
        alpha_arr       = np.array(img.split()[-1])
        norm_alpha_arr  = alpha_arr / 255
        return norm_alpha_arr
        
    def score(function_payload):
            
        try:
            
            from PIL import Image
            canvas_data   = function_payload["input_data"][0]["values"][0] 
            rgba_arr      = getRGBAArr(canvas_data)        
            img           = Image.fromarray(rgba_arr, 'RGBA')
            sm_img        = img.resize((28, 28), Image.LANCZOS)
            alpha_list    = getNormAlphaList(sm_img)           
            alpha_list    = alpha_list.reshape((1,28,28))
            model_payload = {"input_data": [{"values" : alpha_list}]}
                    
            from ibm_watson_machine_learning import APIClient
            client       = APIClient(parms["wml_credentials"])
            client.set.default_space(parms["space_id"])
            model_result = client.deployments.score(parms["model_deployment_id"], model_payload)
                    
            return model_result
        
        except Exception as e:
            
            return {'predictions': [{'values': [repr(e)]}]}


    return score

## Test locale

In [ ]:
func_result = my_deployable_function()({"input_data": [{"values": [sample_payload]}]})
print(func_result)

# Deploy Python Function

Andiamo a fare il deploy andando prima a salvare la funzione come asset nel deployment space, e poi andando a creare un deployment contattabile dall'esterno.

## Store

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'MNIST deployable function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

function_details = client.repository.store_function(meta_props=meta_data, function=my_deployable_function)

function_uid = client.repository.get_function_uid(function_details)

## Deploy

In [ ]:
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "MNIST function deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.TAGS: ['FUNCTION']
}

function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)

In [ ]:
# Prendiamo l'url da contattare per prendere 

function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print(function_deployment_id)
print(function_deployment_endpoint_url)

In [ ]:
# Setup payload

payload = {"input_data": [{"values": [sample_payload]}]}

# Test

Testiamo la funzione tramite API e tramite richiesta HTTPS.

In [ ]:
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + cpdtoken}

payload_scoring = payload

# TODO Change URL
response_scoring = requests.post(f'https://cpd-cpd-instance.itzroks-666000zj44-mn5nm2-4b4a324f027aea19c5cbc0c3275c4656-0000.eu-de.containers.appdomain.cloud/ml/v4/deployments/{function_deployment_id}/predictions?version=2022-04-21', json=payload_scoring, headers=header, verify=False)
print("Scoring response")
print(json.loads(response_scoring.text))